## 2. Lung Classification Tutorial

#### In this course you will learn how to run a deep learning experiment - from loading data, running a model, and deploying your model on a test dataset.

This course will build upon the knowledge gained in the first lesson and will utilize a much larger dataset.

In this course you will build a deep learning model that identifies whether an x-ray of the lungs contains an opacity. The dataset is from a Kaggle challenge.

The dataset comes from the RSNA Pneumonia Detection Challenge (Kaggel API)
        ,
        "The [Radiological Society of North America](http://www.rsna.org/) Pneumonia Detection Challenge: https://www.kaggle.com/c/rsna-pneumonia-detection-challenge",


<img src="images/lesson2_datasetImage.png">



In [ ]:
# Include the mdai module, output current mdai version
!pip install -q --upgrade mdai
import mdai
mdai.__version__

In [ ]:
# Add mdai client
mdai_client = mdai.Client(domain='public.md.ai', access_token="ENTER ACCESS TOKEN")

In [ ]:
p = mdai_client.project('LxR6zdR2', path='./lung-opacity-data')

In [ ]:
p.show_label_groups()

In [ ]:
# this maps label ids to class ids as a dict obj
labels_dict = {'L_ylR0L8':0, # No Lung Opacity 
               'L_DlqEAl':1, # Lung Opacity  
              }

print(labels_dict)
p.set_labels_dict(labels_dict)

In [ ]:
# show dataset ID and label mappings
p.show_datasets() 

### Display label classes

In [ ]:
dataset = p.get_dataset_by_id('D_ao3XWQ')
dataset.prepare()
dataset.show_classes()

In [ ]:
anns = dataset.get_annotations()

while 0:
    # create training dataset 
    train_dataset = p.get_dataset_by_name('TRAIN')
    train_dataset.prepare() 
    train_image_ids = train_dataset.get_image_ids()
    print(len(train_image_ids))


    # create the validation dataset 
    val_dataset = p.get_dataset_by_name('VAL')
    val_dataset.prepare()
    val_image_ids = val_dataset.get_image_ids()
    print(len(val_image_ids))

In [ ]:
train_dataset, val_dataset = mdai.common_utils.train_test_split(dataset, validation_split = 0.98)
val_dataset, test_dataset = mdai.common_utils.train_test_split(val_dataset, validation_split = 0.995)
test_dataset, test_dataset2 = mdai.common_utils.train_test_split(test_dataset, validation_split = 0.90)

In [ ]:
anns = dataset.get_annotations(labels_dict.keys(), verbose=True)

In [ ]:
train_image_ids = train_dataset.get_image_ids()
val_image_ids = val_dataset.get_image_ids()

# visualize a few train images 
mdai.visualize.display_images(train_image_ids[:2], cols=2)
mdai.visualize.display_images(val_image_ids[:2], cols=2)

In [ ]:
import numpy as np
# get image pixel data
pixel_array = mdai.visualize.load_dicom_image(train_image_ids[0], to_RGB=False, rescale=True)
print(np.shape(pixel_array))

In [ ]:
# Import keras module
from keras import applications
from keras.models import Model, Sequential
from keras.layers import Input, Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Define model parameters 
img_width = 128
img_height = 128
epochs = 20

params = {
    'dim': (img_width, img_height),
    'batch_size': 8,
    'n_classes': 2,
    'n_channels': 3,
    'shuffle': True,
}

# Begin Defining Model

Here we build up a very basic CNN architecture (similar in nature to the VGG class of architectures).

Here is where you can feel free to experiment with different architectures and tune the hyperparameters of the network. You should observe differences in training performance, as well as the amount of time required to fully train the network. 

Try changing the number of kernels in the network from 32 down to 16.

For example:

```python
conv1 = Conv2D(16, (3,3), activation = 'relu', padding='same')(inputs)
```

Or changing the size of the filter kernels from 3x3 to 5x5

```python
conv1 = Conv2D(32, (5,5), activation = 'relu', padding='same')(inputs)
```

Or the activation function for the output:

```python
conv1 = Conv2D(32, (3,3), activation = 'tanh', padding='same')(inputs)
```

How do these parameters affect performance and training time?

# Begin Defining Model

In [ ]:
model_test1 = False

# CNN example
if model_test1 is True:
    # Create a basic CNN architecture

    inputs = Input((img_width, img_height, 3))

    # Block1
    conv1 = Conv2D(32, (3,3), activation = 'relu', padding='same')(inputs)
    conv2 = Conv2D(32, (3,3), activation = 'relu', padding='same')(conv1)
    max1 = MaxPooling2D((4,4), strides = (4,4))(conv2)

    # Block2
    conv3 = Conv2D(64, (3,3), activation = 'relu', padding='same')(max1)
    conv4 = Conv2D(64, (3,3), activation = 'relu', padding='same')(conv3)
    max2 = MaxPooling2D((4,4), strides = (4,4))(conv4)

    fcn = Flatten()(max2)
    fcn = Dense(512,activation='relu')(fcn) # was 1028
    fcn = Dropout(0.5)(fcn)
    fcn = Dense(256,activation='relu')(fcn) # was 512
    fcn = Dropout(0.5)(fcn)
    output = Dense(2,activation='softmax')(fcn)

    model = Model(inputs,output)

    model.compile(optimizer=Adam(lr=5.E-6, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
                  loss='categorical_crossentropy', metrics=['accuracy'])

# transfer learning example
if model_test1 is False:
    base_model = applications.mobilenet.MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    model_top  = Sequential()
    model_top.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:], data_format=None))
    model_top.add(Dense(256, activation='relu'))
    model_top.add(Dropout(0.5))
    model_top.add(Dense(2, activation='softmax')) 

    model = Model(inputs=base_model.input, outputs=model_top(base_model.output))

    model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0), 
                  loss='categorical_crossentropy', metrics=['accuracy'])

# End Defining Model

In [ ]:
from mdai.utils import keras_utils

train_generator = keras_utils.DataGenerator(train_dataset, **params)
val_generator = keras_utils.DataGenerator(val_dataset, **params)

In [ ]:
import tensorflow as tf 
config = tf.compat.v1.ConfigProto() #update for tf2 compatibility
config.gpu_options.allow_growth = True

# Train Model

In [ ]:
# Set callback functions to early stop training and save the best model so far
callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, verbose=2),
    ModelCheckpoint(filepath='best_model_lesson2.h5', monitor='val_acc', 
                    save_best_only=True, verbose=2)
]

history = model.fit_generator(
            generator=train_generator,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1,            
            validation_data=val_generator,
            use_multiprocessing=False,
            workers=8)             

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())

plt.figure()
plt.plot(history.history['accuracy'], 'orange', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'blue', label='Validation accuracy')
plt.legend()
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize = 16)
plt.show()

plt.figure()
plt.plot(history.history['loss'], 'red', label='Training loss')
plt.plot(history.history['val_loss'], 'green', label='Validation loss')
plt.legend()
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.show()

# Create the Test dataset

In [ ]:
model.load_weights('best_model_lesson2.h5')
test_dataset.prepare()
print(len(test_dataset.image_ids))

In [ ]:
import numpy as np
#from skimage.transform import resize
from PIL import Image 

for image_id in test_dataset.image_ids[80:100]: 
    
    image = mdai.visualize.load_dicom_image(image_id, to_RGB=True)
    image = Image.fromarray(image)
    image = image.resize((img_width, img_height))
    
    x = np.expand_dims(image, axis=0)    
    y_prob = model.predict(x) 
    y_classes = y_prob.argmax(axis=-1)
    
    title = 'Pred: ' + test_dataset.class_id_to_class_text(y_classes[0]) + ', Prob:' + str(round(y_prob[0][y_classes[0]], 3))
    
    plt.figure()
    plt.title(title)
    plt.imshow(image)
    plt.axis('off')
    
plt.show()

In [ ]:
# ------------ Activation Maps

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras import backend as K
import h5py
from keras.optimizers import SGD

def global_average_pooling(x):
    return K.mean(x, axis = (2,3))

def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

def VGG_like_convolutions():
    model = Sequential()
    
    model.add(ZeroPadding2D((1,1),input_shape=(128,128,3)))
    model.add(Convolution2D(16, (3, 3), activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(16, (3, 3), activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(32, (3, 3), activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(32, (3, 3), activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, (3, 3), activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, (3, 3), activation='relu', name='conv3_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation='relu', name='conv4_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    return model

def get_model_vgg_like():
    model = VGG_like_convolutions()
    
    model.add(Lambda(global_average_pooling, 
              output_shape=global_average_pooling_shape))
    model.add(Dense(2, activation = 'softmax', kernel_initializer='uniform'))
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])
    return model

def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

model = get_model_vgg_like()

# Set callback functions to early stop training and save the best model so far
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=2),
    ModelCheckpoint(filepath='best_model_am2.h5', monitor='val_loss', 
                    save_best_only=True, verbose=2)
]

history = model.fit_generator(
            generator=train_generator,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1, #on a Windows machine you may want to use verbose=0           
            validation_data=val_generator,
            use_multiprocessing=False,
            workers=4) 

In [ ]:
import cv2

def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

def visualize_class_activation_map(model, img):
    model = model
    original_img = img
    width = 128
    height = 128

    #Get the 512 input weights to the softmax.
    class_weights = model.layers[-1].get_weights()[0]
    final_conv_layer = get_output_layer(model, "conv4_2")
    get_output = K.function([model.layers[0].input], \
        [final_conv_layer.output, 
    model.layers[-1].output])
    [conv_outputs, predictions] = get_output([np.float32(img)])
    
    conv_outputs = conv_outputs[0, :, :, :]
    
    #Create the class activation map.
    cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[0:2])
    for i, w in enumerate(class_weights[:, 1]):
            cam += w * conv_outputs[:, :, i]
    #print("predictions", predictions)

    cam = cv2.resize(cam, (height, width))

    return cam



In [ ]:
import numpy as np
from PIL import Image 

for image_id in test_dataset.image_ids[80:100]: 
    
    image = mdai.visualize.load_dicom_image(image_id, to_RGB=True)
    image = Image.fromarray(image)
    image = image.resize((img_width, img_height))
    
    x = np.expand_dims(image, axis=0)    
    y_prob = model.predict(x) 
    y_classes = y_prob.argmax(axis=-1)
    
    title = 'Pred: ' + test_dataset.class_id_to_class_text(y_classes[0]) + ', Prob:' + str(round(y_prob[0][y_classes[0]], 3))
    
    plt.figure()
    plt.title(title)
    plt.imshow(image, cmap='jet')
    plt.axis('off')
    
    # Add the activation map overlay
    cam = visualize_class_activation_map(model, x)
    cam = cam - np.min(cam)
    cam[cam<np.max(cam)/2]=0
    masked = np.ma.masked_where(cam == 0, cam)
    plt.imshow(masked,cmap='jet',alpha=0.3)
    plt.axis('off')
    
plt.show()

In [ ]:
# Generate learning curves for this final network

import matplotlib.pyplot as plt

print(history.history.keys())

plt.figure()
plt.plot(history.history['accuracy'], 'orange', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'blue', label='Validation accuracy')
plt.legend()
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize = 16)
plt.show()

plt.figure()
plt.plot(history.history['loss'], 'red', label='Training loss')
plt.plot(history.history['val_loss'], 'green', label='Validation loss')
plt.legend()
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.show()